In [1]:
import pandas as pd
import requests as rq
import numpy as np
import pyarrow
import os
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
import src.soporte_api as sa
import src.soporte_limpieza as sl
import src.soporte_escrapeo as se
import src.soporte_carga_BBDD as sc

In [2]:
def limpieza_datos(ruta_entrada, ruta_salida,ruta_escrapeo):
    if not os.path.exists(ruta_entrada):
        print(f'Error: No se encontró el archivo {ruta_entrada}')
    else:
        df_raw=pd.read_parquet(ruta_entrada) #importo el fichero original
        df=df_raw.copy() #Hago una copia del df original para seguir trabajando con él
        df = df.drop_duplicates() #Elimino los duplicados
        print(f'El fichero original se ha importando y se han eliminado los duplicados')

        #CONVIERTO LAS COLUMNAS EN FORMATO FECHA
        if "fecha_reserva" in df.columns and "inicio_estancia" in df.columns and "final_estancia" in df.columns:
            col_fechas=["fecha_reserva","inicio_estancia","final_estancia"]
            sl.tran_data(col_fechas, df)
            print(f'Se han convertido las columnas {col_fechas} a formato fecha')

        #CREACIÓN ID CLIENTE ÚNICO:
        #Creo un id para cada cliente basandome la unión del nombre y apellido:
        if "nombre" in df.columns and 'apellido' in df.columns:
            df["nombre_apellido"] = df['nombre'].str.lower() +'.' +df['apellido'].str.lower()
            df.nombre_apellido.unique()
            df_clientes_unicos = pd.DataFrame(df.nombre_apellido.unique())
            df_clientes_unicos['id_cliente_unico'] = None
            for indice in df_clientes_unicos.index:
                df_clientes_unicos.iloc[indice,1] = 'C'+str(indice)+df_clientes_unicos.iloc[indice,0].split('.')[0][0]+df_clientes_unicos.iloc[indice,0].split('.')[1][0]
            df_clientes_unicos=df_clientes_unicos.set_axis(['nombre_apellido', 'id_cliente_unico'], axis=1)

            sl.incorporar_información_df_original(df,df_clientes_unicos, 'nombre_apellido', 'id_cliente_unico', 'id_cliente_unico') #incorporo el id_cliente_unico al df original
            print(f'Se ha creado un id único para cada cliente. Se han creado {len(df_clientes_unicos)} ids de clientes únicos')

        #COMPLETAR VALORES NULOS DE LAS COLUMNAS INICIO Y FINAL ESTANCIA:
        if "inicio_estancia" in df.columns:
            df["inicio_estancia"] = df[df["inicio_estancia"].notna()].inicio_estancia[0]
        if "final_estancia" in df.columns:
            df["final_estancia"] =  df[df["final_estancia"].notna()].final_estancia[0]
        print(f'Se ha completado los nulos de las fechas de inicio estancia con el valor {df["inicio_estancia"][0]} y fin estancia con el valor {df["final_estancia"][0]}')

        #CREACIÓN ID HOTEL ÚNICO:
        #Creo un diccionario nuevo para darle un id a cada hotel propio según el nombre:
        if "competencia" in df.columns and "nombre_hotel" in df.columns and "id_hotel" in df.columns:
            df_no_competencia = df[df.competencia == False]
            lista_nombre_hoteles = df.nombre_hotel.unique().tolist()
            lista_ids_hoteles = df_no_competencia.id_hotel.unique().tolist()
            dict_id_nombre_hotel = {'id_hotel': [], 
                                'nombre_hotel': []}
            for hotel in lista_nombre_hoteles[1:]:
                dict_id_nombre_hotel["nombre_hotel"].append(hotel)
            for id in lista_ids_hoteles:
                dict_id_nombre_hotel["id_hotel"].append(id)
            dict_id_nombre_hotel
            df_id_hoteles_nombre_hoteles = pd.DataFrame(dict_id_nombre_hotel)

            sl.incorporar_información_df_original(df,df_id_hoteles_nombre_hoteles, 'nombre_hotel', 'id_hotel', 'id_hotel',df['competencia']==False ) #incorporo el id_hotel correcto al df original
            print(f'Se han creado los id de hoteles únicos propies. Existen {len(df_id_hoteles_nombre_hoteles)} hoteles')

        #CREACIÓN VALORACIÓN HOTEL:
        #Calculo la valoración para cada hotel como la media de las estrellas:
        if "estrellas" in df.columns and "nombre_hotel" in df.columns:
            df["estrellas"] = df["estrellas"].astype('Int64') #transformo la columna estrellas a tipo entero:
            df_valoracion = df_no_competencia.groupby('nombre_hotel')['estrellas'].mean().round(1) #creo un nuevo df para calcular la valoración de cada hotel según la media de las estrellas
            df_valoracion= df_valoracion.reset_index()

            sl.incorporar_información_df_original(df,df_valoracion, 'nombre_hotel', 'estrellas', 'valoracion',df['competencia']==False ) #incorporo la valoración al df original
            print('Se ha incorporado la información de valoración de los hoteles propios')

        #COMPLETAR VALORES NULOS DEL PRECIO POR NOCHE:
        #Calculo el precio medio de la noche teniendo en cuenta tanto el nombre como la fecha de la reserva:
        if "nombre_hotel" in df.columns and "fecha_reserva" in df.columns:
            df_precios_nulos_hoteles_propios = df_no_competencia[df_no_competencia.precio_noche.isnull()].groupby(['nombre_hotel', 'fecha_reserva'])['id_reserva'].count().reset_index()
            df_precios_nulos_hoteles_propios["precio_noche_medio"] = None
            for indice in range(0,len(df_precios_nulos_hoteles_propios)):
                nombre_hotel= df_precios_nulos_hoteles_propios.iloc[indice,:].nombre_hotel
                fecha_reserva=df_precios_nulos_hoteles_propios.iloc[indice,:].fecha_reserva
                df_precios_nulos_hoteles_propios.iloc[indice,3] = df[df.nombre_hotel == nombre_hotel][df[df.nombre_hotel == nombre_hotel].fecha_reserva == fecha_reserva].precio_noche.mean().round(2)

            #Obtengo los indices de los que tienen precios nulos y son hoteles propios:
            lista_indices_precios_nulos_hoteles_propios = df_no_competencia[df_no_competencia.precio_noche.isnull()].index

            #Sustituyo los precios que falta en el DF original teniendo en cuenta los precios medios calculados:
            for indice in lista_indices_precios_nulos_hoteles_propios:
                fila = df.iloc[indice,:]
                nombre_hotel = fila['nombre_hotel']
                fecha_reserva = fila['fecha_reserva']
                precio_medio = df_precios_nulos_hoteles_propios[(df_precios_nulos_hoteles_propios['nombre_hotel'] == nombre_hotel) & (df_precios_nulos_hoteles_propios['fecha_reserva'] == fecha_reserva)].precio_noche_medio
                df.iloc[indice,10]=precio_medio
            print('Se ha completado los precios por noche nulos existentes, según el precio medio por hotel y fecha de reserva')

            #COMPLETAR LOS DATOS FALTANTES DE LA COMPETENCIA CONSEGUIDOS CON EL ESCRAPEO:
            url = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"
            df_escrapeo = se.escrapeo(url,ruta_escrapeo)
            print(f'Se ha importado la información de los hoteles de la competencia mediante escrapeo. Se ha importado la información de {len(df_escrapeo)} hoteles')

        #Creo un df con el nombre de los hoteles de la competencia y el id del hotel:
        if "competencia" in df.columns and "nombre_hotel" in df.columns and "id_hotel" in df.columns:
            df_competencia = df[df.competencia == True] 
            values = list(df_competencia.id_hotel.drop_duplicates())
            keys = list(df_escrapeo.nombre_hotel.unique())
            diccionario_idhotel_nombrehotel = dict(zip(keys,values))
            df_idhotel_nombrehotel_competencia = pd.DataFrame(list(diccionario_idhotel_nombrehotel.items()), columns=['nombre_hotel', 'id_hotel'])

            sl.incorporar_información_df_original(df_escrapeo,df_idhotel_nombrehotel_competencia, 'nombre_hotel', 'id_hotel', 'id_hotel') #Incorpor el id del hotel al df del escrapeo
            print(f'Se ha asignado el nombre del hotel de la competencia a los {len(df_idhotel_nombrehotel_competencia)} id de hoteles existentes.')
            #Renombro las columnas del df_escrapeo:
            df_escrapeo=df_escrapeo.rename(columns={
                'rating': 'valoracion',
                'fecha_escrapeo': 'fecha_reserva'
            })

            #Incorporo la información del df del escrapeo al df original
            for columna in df_escrapeo.columns[:-1]:
                sl.incorporar_información_df_original(df,df_escrapeo, 'id_hotel', columna, columna,df['competencia']==True )
            print(f'Se ha incorporado la información de las columnas {df_escrapeo.columns[:-1].to_list()} de los hoteles de la competencia en el fichero original')

        #ELIMINO LAS COLUMNAS QUE NO NECESITO:
        if "id_cliente" in df.columns and "estrelas" in df.columns and "nombre_apellido" in df.columns:
            df.drop(columns=['id_cliente', 'estrellas','nombre_apellido'],inplace=True)
            print(f'Se han eliminado las columnas que no son necesarias')

        #GUARDO EL DF COMO PICKLE:
        df.to_pickle(ruta_salida)
        print(f'Se ha guardado el fichero limpio en la ruta {ruta_salida}')

In [3]:
ruta_entrada = '../data/reservas_hoteles.parquet'
ruta_salida = '../data/datos_parquet_limpios.pkl'
ruta_escrapeo='../data/scrapeo.pkl'
limpieza_datos(ruta_entrada, ruta_salida,ruta_escrapeo)


El fichero original se ha importando y se han eliminado los duplicados
Se han convertido las columnas ['fecha_reserva', 'inicio_estancia', 'final_estancia'] a formato fecha
Se ha creado un id único para cada cliente. Se han creado 14905 ids de clientes únicos
Se ha completado los nulos de las fechas de inicio estancia con el valor 2025-03-01 00:00:00 y fin estancia con el valor 2025-03-02 00:00:00
Se han creado los id de hoteles únicos propies. Existen 19 hoteles
Se ha incorporado la información de valoración de los hoteles propios
Se ha completado los precios por noche nulos existentes, según el precio medio por hotel y fecha de reserva
Se ha importado la información de los hoteles de la competencia mediante escrapeo. Se ha importado la información de 10 hoteles
Se ha asignado el nombre del hotel de la competencia a los 10 id de hoteles existentes.
Se ha incorporado la información de las columnas ['nombre_hotel', 'valoracion', 'precio_noche', 'fecha_reserva', 'ciudad'] de los hoteles 

c:\Formación\DataAnalytics\BootCampDataAnalyst\Proyectos\ProyectoETL\Proyecto_ETL\notebooks\..\src\soporte_limpieza.py:86: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['268' '110' '181' ... '143' '119' '110']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataframe_a_rellenar.loc[filtro_df_original,columna_a_rellenar] = dataframe_a_rellenar[columna_union].map(diccionario_creado)
